In [1]:
# Random Imports
import os
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/SeBench')

# Regular schema dictates that we put DATAPATH
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ['WANDB_NOTEBOOK_NAME'] = 'train.ipynb'

# IonPy imports
from ionpy.util import Config

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [2]:
%%yaml default_cfg 

experiment:
  seed: '?' 
  val_first: False 
  torch_compile: True 
  torch_mixed_precision: False
    
dataloader:
  batch_size: '?' 
  num_workers: '?'
  pin_memory: True 

optim: 
  _class: torch.optim.Adam
  lr: 1.0e-4 
  
train:
  epochs: 300 
  eval_freq: 10 

# Used for additional data experiment.
data:
  train_kwargs:
    split: 'train'
  val_kwargs: 
    split: 'val'
  
loss_func: 
  _class: sebench.losses.SoftDiceLoss
  from_logits: True
  batch_reduction: 'mean' 
  ignore_empty_labels: False 

<IPython.core.display.Javascript object>

In [3]:
%%yaml log_cfg

log:
  root: '?'
  checkpoint_freq: 10 
  metrics:
    dice_score:
      _fn: ionpy.metrics.dice_score
      batch_reduction: 'mean' 
      ignore_empty_labels: False 
      from_logits: True

<IPython.core.display.Javascript object>

In [4]:
%%yaml model_cfg  

model:
  _class: sebench.models.Segmenter

<IPython.core.display.Javascript object>

In [5]:
%%yaml callbacks_cfg

callbacks:
  step:
    - sebench.callbacks.ShowPredictions
  epoch:
    - sebench.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: dice_score
        phase: val

<IPython.core.display.Javascript object>

In [6]:
# %%yaml aug_cfg

# augmentations:
#     spatial:
#         max_translation: 5.0
#         max_rotation: 5.0
#         max_scaling: 1.1
#         warp_integrations: 5
#         warp_smoothing_range: [10, 20]
#         warp_magnitude_range: [1, 3]
#         affine_probability: 0.5
#         warp_probability: 0.5
#     visual:
#         use_mask: False 
#         added_noise_max_sigma: 0.01
#         gamma_scaling_max: 0.1
#         bias_field_probability: 0.5
#         gamma_scaling_probability: 0.5
#         added_noise_probability: 0.5

In [7]:
%%yaml experiment_cfg 

group: "WBC_Segmenter"

experiment:
    seed: 40
    seed_range: 1
    torch_compile: False 

data:
    _class: "sebench.datasets.Segment2D"
    task: "WBC/CV/EM/0"
    resolution: 64

dataloader:
    batch_size: 1
    num_workers: 1

<IPython.core.display.Javascript object>

In [8]:
from sebench.experiment.utils import get_training_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([model_cfg, log_cfg, callbacks_cfg])
# base_cfg = Config(default_cfg).update([model_cfg, log_cfg, callbacks_cfg, aug_cfg])

# Get the different experiment cfg yamls.
updated_base_cfg, train_cfgs = get_training_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg
)

In [ ]:
len(train_cfgs)

1

## Running Jobs

In [10]:
####### FOR DEBUGGIN
from sebench.experiment.runner import run_exp
from sebench.experiment.train import SegTrainExperiment

run_exp(
    config=train_cfgs[0], 
    experiment_class=SegTrainExperiment,
    run_name='debug',
    show_examples=True,
    track_wandb=False,
    gpu='0',
)

Set seed: 40


/storage/vbutoi/projects/ionpy/util/libcheck.py:49: UserWarning: Intel MKL extensions not available for NumPy
  warn("Intel MKL extensions not available for NumPy")
/storage/vbutoi/projects/ionpy/util/libcheck.py:57: UserWarning: Using slow Pillow instead of Pillow-SIMD
  warn("Using slow Pillow instead of Pillow-SIMD")
/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/storage/vbutoi/projects/ionpy/util/libcheck.py:59: UserWarning: PyTorch cannot find a valid GPU device, check CUDA_VISIBLE_DEVICES
  warn("PyTorch cannot find a valid GPU device, check CUDA_VISIBLE_DEVICES")
/local/vbutoi/envs/UniverSegT

Running SegTrainExperiment("/storage/vbutoi/scratch/SeBench/training/debug/20241219_231927-74E4-96831b1b57528fcb57af6480c2e1fd52")
---
callbacks:
  epoch:
  - sebench.callbacks.WandbLogger
  - ionpy.callbacks.ETA
  - ionpy.callbacks.JobProgress
  - ionpy.callbacks.TerminateOnNaN
  - ionpy.callbacks.PrintLogged
  - ionpy.callbacks.ModelCheckpoint:
      monitor: dice_score
      phase: val
  step:
  - sebench.callbacks.ShowPredictions
data:
  _class: sebench.datasets.Segment2D
  resolution: 64
  task: WBC/CV/EM/0
  train_kwargs:
    split: train
  val_kwargs:
    split: val
dataloader:
  batch_size: 1
  num_workers: 1
  pin_memory: true
experiment:
  seed: 40
  torch_compile: false
  torch_mixed_precision: false
  val_first: false
log:
  checkpoint_freq: 10
  metrics:
    dice_score:
      _fn: ionpy.metrics.dice_score
      batch_reduction: mean
      from_logits: true
      ignore_empty_labels: false
  root: /storage/vbutoi/scratch/SeBench/training/debug
  show_examples: true
  uuid: 

wandb: Currently logged in as: vbutoi. Use `wandb login --relogin` to force relogin


Start training epoch 0.
flat x: torch.Size([1, 64, 64])
proj x: torch.Size([1, 64, 16])
Split head: torch.Size([1, 64, 8, 2])
Split head: torch.Size([1, 64, 8, 2])
Split head: torch.Size([1, 64, 8, 2])
qkv: torch.Size([1, 8, 64, 2]) torch.Size([1, 8, 64, 2]) torch.Size([1, 8, 64, 2])
prod: torch.Size([1, 8, 64, 64])
z: torch.Size([1, 8, 64, 2])
Combin head shape: torch.Size([1, 8, 64, 2])
Preview shape: torch.Size([1, 64, 8, 2])


RuntimeError: shape '[1, 8, 16]' is invalid for input of size 1024

In [ ]:
# # FOR SUBMISSION
# from sebench.experiment.utils import run_exp
# from sebench.experiment.train import SegTrainExperiment

# submit_exps(
#     group="training",
#     base_cfg=updated_base_cfg,
#     exp_cfg=experiment_cfg,
#     config_list=train_cfgs,
#     experiment_class=SegTrainExperiment,
#     track_wandb=True,
#     available_gpus=['0', '1', '2', '3', '4', '5', '6', '7'],
# )